# part 1

In [71]:
Dictionary = {}
Begining = {}
Middle = {}
Ending = {}
WordFreq = {}

## 1.Dictionary

In [72]:
dict_file = open('dict.txt', 'r')
dict_Lines = dict_file.readlines()
 
count = 0
for line in dict_Lines:
    count += 1
    splited = line[:-1].split(" ")
    if len(splited) != 2: continue
    key = splited[0]
    value = splited[1]
    Dictionary[key] = value
    
print(f"number of lines {count}")

number of lines 689


In [73]:
print(Dictionary["tokyo"])
print(Dictionary["voa"])
print(Dictionary["sos"])
print(Dictionary["zahra"])

توکیو
وی‌او‌ای
سس
زهرا


## 2.Begining

In [74]:
beg_file = open('beginning.txt', 'r')
beg_Lines = beg_file.readlines()
 
count = 0
for line in beg_Lines:
    count += 1
    splited = line[:-1].split(" ")
    key = splited[0]
    values = splited[1:]
    Begining[key] = values
    
print(f"number of lines {count}")

number of lines 38


In [75]:
print(Begining["kha"])
print(Begining["sh"])
print(Begining["j"])
print(Begining["ch"])

['خا', 'خوا', 'خ']
['ش']
['ج', 'ژ']
['چ']


## 3.Middle

In [76]:
mid_file = open('middle.txt', 'r')
mid_Lines = mid_file.readlines()
 
count = 0
for line in mid_Lines:
    count += 1
    splited = line[:-1].split(" ")
    key = splited[0]
    values = splited[1:]
    Middle[key] = values
    
print(f"number of lines {count}")

number of lines 45


In [77]:
print(Middle["sh"])
print(Middle["i"])
print(Middle["ck"])
print(Middle["a"])

['ش']
['ی', 'یع', 'ئی', 'عی']
['ک']
['nothing', 'ع', 'ا', 'اع', 'أ', 'ؤ', 'ؤا', 'ئا', 'ئ']


## 4.Ending

In [78]:
end_file = open('ending.txt', 'r')
end_Lines = end_file.readlines()
 
count = 0
for line in end_Lines:
    count += 1
    splited = line[:-1].split(" ")
    key = splited[0]
    values = splited[1:]
    Ending[key] = values
    
print(f"number of lines {count}")

number of lines 47


In [79]:
print(Ending["sh"])
print(Ending["i"])
print(Ending["ck"])
print(Ending["a"])

['ش']
['ی', 'یع']
['ک']
['ا', 'اع', 'nothing']


## 5.WordFreq

In [80]:
freq_file = open('persian-wikipedia.txt', 'r')
freq_Lines = freq_file.readlines()
 
min_freq = 4
count = 0
for line in freq_Lines:
    count += 1
    splited = line[:-1].split("\t")
    word = splited[0]
    freq = int(splited[1]) 
    if freq < min_freq:
        break
    if word not in WordFreq.keys():
        WordFreq[word] = freq

    
print(f"number of lines {count}")

number of lines 111302


In [81]:
print(WordFreq["یک"])
print(WordFreq["جمعیت"])
print(WordFreq["گودترین"])
print(WordFreq["لطفا"])

84193
24228
5
87


# part 2

In [82]:
import itertools
import re

sep_regex = re.compile(r'[ \-_~!@#%$^&*\(\)\[\]\{\}/\:;"|,./?`]')

In [83]:
def f2p_word_internal(word, original_word):
    # this function receives the word as separate letters
    persian = []
    for i, letter in enumerate(word):
        if i == 0:
            converter = Begining
        elif i == len(word) - 1:
            converter = Ending
        else:
            converter = Middle
        conversions = converter.get(letter)
        if conversions == None:
            return [(''.join(original_word), 0.0)]
        else:
            conversions = ['' if i == 'nothing' else i for i in conversions]
        persian.append(conversions)

    alternatives = itertools.product(*persian)
    alternatives = [''.join(i) for i in alternatives]

    alternatives = [(i, WordFreq[i]) if i in WordFreq else (i, 0)
                    for i in alternatives]

    if len(alternatives) > 0:
        max_freq = max(freq for _, freq in alternatives)
        alternatives = [(w, float(freq / max_freq)) if freq != 0 else (w, 0.0)
                        for w, freq in alternatives]
    else:
        alternatives = [(''.join(word), 1.0)]

    return alternatives

In [84]:
def variations(word):
    """Create variations of the word based on letter combinations like oo,
sh, etc."""

    if word == 'a':
        return [['A']]
    elif len(word) == 1:
        return [[word[0]]]
    elif word == 'aa':
        return [['A']]
    elif word == 'ee':
        return [['i']]
    elif word == 'ei':
        return [['ei']]
    elif word in ['oo', 'ou']:
        return [['u']]
    elif word == 'kha':
        return [['kha'], ['kh', 'a']]
    elif word in ['kh', 'gh', 'ch', 'sh', 'zh', 'ck']:
        return [[word]]
    elif word in ["'ee", "'ei"]:
        return [["'i"]]
    elif word in ["'oo", "'ou"]:
        return [["'u"]]
    elif word in ["a'", "e'", "o'", "i'", "u'", "A'"]:
        return [[word[0] + "'"]]
    elif word in ["'a", "'e", "'o", "'i", "'u", "'A"]:
        return [["'" + word[1]]]
    elif len(word) == 2 and word[0] == word[1]:
        return [[word[0]]]

    if word[:2] == 'aa':
        return [['A'] + i for i in variations(word[2:])]
    elif word[:2] == 'ee':
        return [['i'] + i for i in variations(word[2:])]
    elif word[:2] in ['oo', 'ou']:
        return [['u'] + i for i in variations(word[2:])]
    elif word[:3] == 'kha':
        return \
            [['kha'] + i for i in variations(word[3:])] + \
            [['kh', 'a'] + i for i in variations(word[3:])] + \
            [['k', 'h', 'a'] + i for i in variations(word[3:])]
    elif word[:2] in ['kh', 'gh', 'ch', 'sh', 'zh', 'ck']:
        return \
            [[word[:2]] + i for i in variations(word[2:])] + \
            [[word[0]] + i for i in variations(word[1:])]
    elif word[:2] in ["a'", "e'", "o'", "i'", "u'", "A'"]:
        return [[word[:2]] + i for i in variations(word[2:])]
    elif word[:3] in ["'ee", "'ei"]:
        return [["'i"] + i for i in variations(word[3:])]
    elif word[:3] in ["'oo", "'ou"]:
        return [["'u"] + i for i in variations(word[3:])]
    elif word[:2] in ["'a", "'e", "'o", "'i", "'u", "'A"]:
        return [[word[:2]] + i for i in variations(word[2:])]
    elif len(word) >= 2 and word[0] == word[1]:
        return [[word[0]] + i for i in variations(word[2:])]
    else:
        return [[word[0]] + i for i in variations(word[1:])]

    

In [85]:
def f2p_word(word, max_word_size=15, cutoff=3):
    """Convert a single word from Finglish to Persian.

    max_word_size: Maximum size of the words to consider. Words larger
    than this will be kept unchanged.

    cutoff: The cut-off point. For each word, there could be many
    possibilities. By default 3 of these possibilities are considered
    for each word. This number can be changed by this argument.

    """

    original_word = word
    word = word.lower()

    c = Dictionary.get(word)
    if c:
        return [(c, 1.0)]

    if word == '':
        return []
    elif len(word) > max_word_size:
        return [(original_word, 1.0)]

    results = []
    for w in variations(word):
        #print(w)
        results.extend(f2p_word_internal(w, original_word))

    # sort results based on the confidence value
    results.sort(key=lambda r: r[1], reverse=True)

    # return the top three results in order to cut down on the number
    # of possibilities.
    return results[:cutoff]

In [86]:
def f2p_list(phrase, max_word_size=15, cutoff=3):
    """Convert a phrase from Finglish to Persian.

    phrase: The phrase to convert.

    max_word_size: Maximum size of the words to consider. Words larger
    than this will be kept unchanged.

    cutoff: The cut-off point. For each word, there could be many
    possibilities. By default 3 of these possibilities are considered
    for each word. This number can be changed by this argument.

    Returns a list of lists, each sub-list contains a number of
    possibilities for each word as a pair of (word, confidence)
    values.

    """

    # split the phrase into words
    results = [w for w in sep_regex.split(phrase) if w]

    # return an empty list if no words
    if results == []:
        return []

    # convert each word separately
    results = [f2p_word(w, max_word_size, cutoff) for w in results]

    return results

In [87]:
def f2p(phrase, max_word_size=15, cutoff=3):
    """Convert a Finglish phrase to the most probable Persian phrase.

    """

    results = f2p_list(phrase, max_word_size, cutoff)
    #for i in results:
    #    print(i)
    return ' '.join(i[0][0] for i in results)

# part 3

In [88]:
print(f2p("2 ta yar baraye volleyball ehtiaj darim saat 2 salon nobakht"))

2 تا یار برای ولیبل احتیاج داریم سات 2 سالن نوبخت


In [89]:
print(f2p("saat"))

سات


In [90]:
print(f2p("lotfan"))

لتفن


In [91]:
print(f2p("cheraq"))

چراغ


In [92]:
print(f2p("kardehim"))

کردهیم


In [93]:
print(f2p("khahar"))

خواهر


In [94]:
input_text = "Ba zohoore modelhaye zabani bozorg mitavan dar zamane kam khorooji monaseb daryaft kard."
output_text = "با ظهور مدل\u200cهای زبانی بزرگ می توان در زمان کم خروجی مناسب دریافت کرد."

result = f2p(input_text)
print(result)
print(output_text)


با ظهور مدلهای زبانی بزرگ میتوان در زمان کم خروجی مناسب دریافت کرد
با ظهور مدل‌های زبانی بزرگ می توان در زمان کم خروجی مناسب دریافت کرد.


In [95]:
text = "misheh ino farsi koni?"
print(f2p(text))

میشه این فارسی کنی


In [96]:
text = "inke eshkal dare!"
print(f2p(text))

اینکه اشکال در


In [97]:
text = "Ino por konin lotfan"
print(f2p(text))

این پر کونین لتفن


In [98]:
text = "age emshab nashe ta farada hatman mishe."
print(f2p(text))

اگه امشب نش تا فردا حتماً میش


In [99]:
text = "sabzo sefido ghermez"
print(f2p(text))

سبز سفید قرمز


In [100]:
print(f2p("Albte fk nakonam in tor bashe."))

البته فک نکونام این طور باش


# part 4

In [104]:
fingilish_sentences = [
    "Emrooz hava besyar delpazir bood.",
    "Madaram ghaza-ye khoshmaze-i baraye sham dorost kardeh ast.",
    "Farda be kuhpeymayi khahim raft.",
    "Ketabe jadidi ke kharideham besyar jaleb ast.",
    "Doost daram vaghti hava abri ast be park beravam.",
    "Khaneman ra taze rang kardehim.",
    "Har rooz sobh yek fenjan chai minusham.",
    "Gorbehaman asheghe bazi ba kamva ast.",
    "Pedaram hamishe mara be varzesh tashvigh mikonad.",
    "Hafte ayande be didare doostanam miravam.",
    "Yadgiriye zabane jadid mitavanad tajrobe jazabi bashad.",
    "Emrooz ba khanevade be sinema raftim.",
    "Man be golhaye hayatman rasidgi mikonam.",
    "Har shab ghabl az khob kami motalee mikonam.",
    "Emtehan hafte ayande kheyli mohem ast.",
    "Az tamashaye ghorube aftab lezzat mibaram.",
    "Behtarin zaman baraye piaderavi sobh zood ast.",
    "Khaharam naghashihaye zibai mikeshad.",
    "Karhaye honari ra kheyli doost daram.",
    "Baraye rooze tavallode doostam yek hediye aali kharidam."
]

In [105]:
persian_sentences = [
    "امروز هوا بسیار دلپذیر بود.",
    "مادرم غذای خوشمزه‌ای برای شام درست کرده است.",
    "فردا به کوه‌پیمایی خواهیم رفت.",
    "کتاب جدیدی که خریده‌ام بسیار جالب است.",
    "دوست دارم وقتی هوا ابری است به پارک بروم.",
    "خانه‌مان را تازه رنگ کرده‌ایم.",
    "هر روز صبح یک فنجان چای می‌نوشم.",
    "گربه‌مان عاشق بازی با کاموا است.",
    "پدرم همیشه مرا به ورزش تشویق می‌کند.",
    "هفته آینده به دیدار دوستانم می‌روم.",
    "یادگیری زبان جدید می‌تواند تجربه جذابی باشد.",
    "امروز با خانواده به سینما رفتیم.",
    "من به گل‌های حیاط‌مان رسیدگی می‌کنم.",
    "هر شب قبل از خواب کمی مطالعه می‌کنم.",
    "امتحان هفته آینده خیلی مهم است.",
    "از تماشای غروب آفتاب لذت می‌برم.",
    "بهترین زمان برای پیاده‌روی صبح زود است.",
    "خواهرم نقاشی‌های زیبایی می‌کشد.",
    "کارهای هنری را خیلی دوست دارم.",
    "برای روز تولد دوستم یک هدیه عالی خریدم."
]

In [106]:
for f_sentence, p_sentence in zip(fingilish_sentences, persian_sentences):
    print(f2p(f_sentence))
    print(p_sentence)
    print("#"*30)

امروز هوا بسیار دلپذیر بود
امروز هوا بسیار دلپذیر بود.
##############################
مادرم غذا یه خوشمزه ای برای شام درست کرده است
مادرم غذای خوشمزه‌ای برای شام درست کرده است.
##############################
فردا به کوهپیمایی خواهیم رفت
فردا به کوه‌پیمایی خواهیم رفت.
##############################
کتاب جدیدی که خاریدهم بسیار جلب است
کتاب جدیدی که خریده‌ام بسیار جالب است.
##############################
دوست دارم وقتی هوا عبری است به پارک بروم
دوست دارم وقتی هوا ابری است به پارک بروم.
##############################
خانمان را تازه رنگ کردهیم
خانه‌مان را تازه رنگ کرده‌ایم.
##############################
هر روز صبح یک فنجان چای مینوشم
هر روز صبح یک فنجان چای می‌نوشم.
##############################
گربهمن عشق بازی با کاموا است
گربه‌مان عاشق بازی با کاموا است.
##############################
پدرم همیشه مرا به ورزش تشویق میکند
پدرم همیشه مرا به ورزش تشویق می‌کند.
##############################
هفت آینده به دیدار دوستانم میروم
هفته آینده به دیدار دوستانم می‌روم.
##############################
ید